# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [6]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, 
                         activation = "relu", 
                         input_shape = (X_train.shape[1],)))

# Hidden layer with 50 units

baseline_model.add(layers.Dense(50, activation = "relu"))
# Output layer
baseline_model.add(layers.Dense(1, activation = "linear"))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [9]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train[numeric_columns]))

# Transform validate and test data
X_val_scaled  = pd.DataFrame(ss_X.transform(X_val[numeric_columns]))
X_test_scaled = pd.DataFrame(ss_X.transform(X_test[numeric_columns]))

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [10]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [12]:
# Train the model 
normalized_input_model.fit(X_train, y_train,
                           epochs = 150, batch_size = 32,
                           validation_data = (X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 1ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [13]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled  = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [14]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model

normalized_model.fit(X_train, y_train_scaled,
                    epochs = 150, batch_size = 32,
                    validation_data = (X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.5117 - mse: 0.5117 - val_loss: 0.2658 - val_mse: 0.2658
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2252 - mse: 0.2252 - val_loss: 0.2162 - val_mse: 0.2162
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1920 - mse: 0.1920 - val_loss: 0.1976 - val_mse: 0.1976
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1657 - mse: 0.1657 - val_loss: 0.1797 - val_mse: 0.1797
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1510 - mse: 0.1510 - val_loss: 0.1756 - val_mse: 0.1756
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1395 - mse: 0.1395 - val_loss: 0.1702 - val_mse: 0.1702
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1260 - mse: 0.1260 - val_loss: 0.1651 - val_mse: 0.1651
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1182 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1887 - val_mse: 0.1887
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.1881 - val_mse: 0.1881
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.1865 - val_mse: 0.1865
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1876 - val_mse: 0.1876
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1895 - val_mse: 0.1895
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1887 - val_mse: 0.1887
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0219 - mse: 0.0219 - val_loss: 0.1898 - val_mse: 0.1898
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0221 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1997 - val_mse: 0.1997
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1994 - val_mse: 0.1994
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.2007 - val_mse: 0.2007
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.2044 - val_mse: 0.2044
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1995 - val_mse: 0.1995
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.2025 - val_mse: 0.2025
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.2012 - val_mse: 0.2012
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 650us/step - loss: 0.0080 - mse: 0.0080


[0.00796041265130043, 0.00796041265130043]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [16]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 692us/step - loss: 0.2051 - mse: 0.2051


[0.2051234394311905, 0.2051234394311905]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [21]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
RMSE = mean_squared_error(y_val, y_val_pred, squared = False)
RMSE

35590.91210505595

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [23]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation = "relu", 
                          kernel_initializer = "he_normal",
                          input_shape = (X_train.shape[1],)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.5365 - mse: 0.5365 - val_loss: 0.1936 - val_mse: 0.1936
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2515 - mse: 0.2515 - val_loss: 0.1565 - val_mse: 0.1565
Epoch 3/150
33/33 [==============================] - 0s 1ms/step - loss: 0.2123 - mse: 0.2123 - val_loss: 0.1445 - val_mse: 0.1445
Epoch 4/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1921 - mse: 0.1921 - val_loss: 0.1493 - val_mse: 0.1493
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1647 - mse: 0.1647 - val_loss: 0.1509 - val_mse: 0.1509
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1447 - mse: 0.1447 - val_loss: 0.1479 - val_mse: 0.1479
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1340 - mse: 0.1340 - val_loss: 0.1460 - val_mse: 0.1460
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1224 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0258 - mse: 0.0258 - val_loss: 0.1893 - val_mse: 0.1893
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0252 - mse: 0.0252 - val_loss: 0.1906 - val_mse: 0.1906
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0249 - mse: 0.0249 - val_loss: 0.1882 - val_mse: 0.1882
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0243 - mse: 0.0243 - val_loss: 0.1898 - val_mse: 0.1898
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1935 - val_mse: 0.1935
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0237 - mse: 0.0237 - val_loss: 0.1885 - val_mse: 0.1885
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1911 - val_mse: 0.1911
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0230 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.1981 - val_mse: 0.1981
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1972 - val_mse: 0.1972
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1965 - val_mse: 0.1965
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1969 - val_mse: 0.1969
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1990 - val_mse: 0.1990
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1977 - val_mse: 0.1977
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1970 - val_mse: 0.1970
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 657us/step - loss: 0.0082 - mse: 0.0082


[0.008214985020458698, 0.008214985020458698]

Evaluate the model (`he_model`) on validate data (`X_val` and `y_val_scaled`) 

In [26]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 783us/step - loss: 0.1979 - mse: 0.1979


[0.19792889058589935, 0.19792889058589935]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [27]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation = "relu",
                            kernel_initializer = "lecun_normal",
                            input_shape = (X_train.shape[1],)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.4230 - mse: 0.4230 - val_loss: 0.1823 - val_mse: 0.1823
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2504 - mse: 0.2504 - val_loss: 0.1546 - val_mse: 0.1546
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1969 - mse: 0.1969 - val_loss: 0.1418 - val_mse: 0.1418
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1702 - mse: 0.1702 - val_loss: 0.1325 - val_mse: 0.1325
Epoch 5/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1491 - mse: 0.1491 - val_loss: 0.1198 - val_mse: 0.1198
Epoch 6/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1404 - mse: 0.1404 - val_loss: 0.1144 - val_mse: 0.1144
Epoch 7/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1252 - mse: 0.1252 - val_loss: 0.1125 - val_mse: 0.1125
Epoch 8/150
33/33 [==============================] - 0s 1ms/step - loss: 0.1170 - m

33/33 [==============================] - 0s 1ms/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1407 - val_mse: 0.1407
Epoch 64/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1425 - val_mse: 0.1425
Epoch 65/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0213 - mse: 0.0213 - val_loss: 0.1436 - val_mse: 0.1436
Epoch 66/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1429 - val_mse: 0.1429
Epoch 67/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1427 - val_mse: 0.1427
Epoch 68/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0206 - mse: 0.0206 - val_loss: 0.1427 - val_mse: 0.1427
Epoch 69/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0200 - mse: 0.0200 - val_loss: 0.1457 - val_mse: 0.1457
Epoch 70/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0197 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1508 - val_mse: 0.1508
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1509 - val_mse: 0.1509
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1513 - val_mse: 0.1513
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1503 - val_mse: 0.1503
Epoch 129/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1521 - val_mse: 0.1521
Epoch 130/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1527 - val_mse: 0.1527
Epoch 131/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1510 - val_mse: 0.1510
Epoch 132/150
33/33 [==============================] - 0s 1ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [28]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 674us/step - loss: 0.0088 - mse: 0.0088


[0.008768126368522644, 0.008768126368522644]

Evaluate the model (`lecun_model`) on validate data (`X_val` and `y_val_scaled`) 

In [29]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 712us/step - loss: 0.1532 - mse: 0.1532


[0.15321286022663116, 0.15321286022663116]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [31]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer = "rmsprop", loss = "mse", metrics = ["mse"])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3343 - mse: 0.3343 - val_loss: 0.1196 - val_mse: 0.1196
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1779 - mse: 0.1779 - val_loss: 0.1412 - val_mse: 0.1412
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1327 - mse: 0.1327 - val_loss: 0.1018 - val_mse: 0.1018
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0962 - mse: 0.0962 - val_loss: 0.1278 - val_mse: 0.1278
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0937 - mse: 0.0937 - val_loss: 0.1127 - val_mse: 0.1127
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0647 - mse: 0.0647 - val_loss: 0.1218 - val_mse: 0.1218
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0557 - mse: 0.0557 - val_loss: 0.1334 - val_mse: 0.1334
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0474 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0073 - mse: 0.0073 - val_loss: 0.1586 - val_mse: 0.1586
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0093 - mse: 0.0093 - val_loss: 0.1461 - val_mse: 0.1461
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1382 - val_mse: 0.1382
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1389 - val_mse: 0.1389
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.1282 - val_mse: 0.1282
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0075 - mse: 0.0075 - val_loss: 0.1314 - val_mse: 0.1314
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0072 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0049 - mse: 0.0049 - val_loss: 0.1176 - val_mse: 0.1176
Epoch 126/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.1162 - val_mse: 0.1162
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.1269 - val_mse: 0.1269
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0039 - mse: 0.0039 - val_loss: 0.1079 - val_mse: 0.1079
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.1122 - val_mse: 0.1122
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.1139 - val_mse: 0.1139
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [32]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 677us/step - loss: 0.0028 - mse: 0.0028


[0.0027529627550393343, 0.0027529627550393343]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 747us/step - loss: 0.1118 - mse: 0.1118


[0.11178913712501526, 0.11178913712501526]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [34]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer = "Adam", loss = "mse", metrics = ["mse"])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 4ms/step - loss: 0.3832 - mse: 0.3832 - val_loss: 0.1305 - val_mse: 0.1305
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1599 - mse: 0.1599 - val_loss: 0.1216 - val_mse: 0.1216
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1067 - mse: 0.1067 - val_loss: 0.1150 - val_mse: 0.1150
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0821 - mse: 0.0821 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0588 - mse: 0.0588 - val_loss: 0.1167 - val_mse: 0.1167
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0474 - mse: 0.0474 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0359 - mse: 0.0359 - val_loss: 0.1388 - val_mse: 0.1388
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0294 - m

Epoch 63/150
33/33 [==============================] - 0s 2ms/step - loss: 2.8772e-04 - mse: 2.8772e-04 - val_loss: 0.1169 - val_mse: 0.1169
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 3.1232e-04 - mse: 3.1232e-04 - val_loss: 0.1156 - val_mse: 0.1156
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 3.2732e-04 - mse: 3.2732e-04 - val_loss: 0.1165 - val_mse: 0.1165
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 3.2007e-04 - mse: 3.2007e-04 - val_loss: 0.1137 - val_mse: 0.1137
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 3.2825e-04 - mse: 3.2825e-04 - val_loss: 0.1170 - val_mse: 0.1170
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 3.3012e-04 - mse: 3.3012e-04 - val_loss: 0.1143 - val_mse: 0.1143
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 3.9314e-04 - mse: 3.9314e-04 - val_loss: 0.1177 - val_mse: 0.1177
Epoch 70/150
33/33 [

33/33 [==============================] - 0s 2ms/step - loss: 8.0730e-05 - mse: 8.0730e-05 - val_loss: 0.1002 - val_mse: 0.1002
Epoch 124/150
33/33 [==============================] - 0s 1ms/step - loss: 6.6349e-05 - mse: 6.6349e-05 - val_loss: 0.0993 - val_mse: 0.0993
Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 7.1356e-05 - mse: 7.1356e-05 - val_loss: 0.1000 - val_mse: 0.1000
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 6.5658e-05 - mse: 6.5658e-05 - val_loss: 0.0988 - val_mse: 0.0988
Epoch 127/150
33/33 [==============================] - 0s 1ms/step - loss: 5.9304e-05 - mse: 5.9304e-05 - val_loss: 0.1002 - val_mse: 0.1002
Epoch 128/150
33/33 [==============================] - 0s 1ms/step - loss: 5.9500e-05 - mse: 5.9500e-05 - val_loss: 0.0991 - val_mse: 0.0991
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 5.9513e-05 - mse: 5.9513e-05 - val_loss: 0.1000 - val_mse: 0.1000
Epoch 130/150
33/33 [======

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [35]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 657us/step - loss: 0.0033 - mse: 0.0033


[0.003275700379163027, 0.003275700379163027]

Evaluate the model (`adam_model`) on training data (`X_val` and `y_val_scaled`) 

In [36]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 677us/step - loss: 0.0999 - mse: 0.0999


[0.09989020228385925, 0.09989020228385925]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [41]:
# Evaluate the best model on test data
# They used rmsprop_model but for me adam_model looks better

print(adam_model.evaluate(X_test, y_test_scaled))
print()
print(rmsprop_model.evaluate(X_test, y_test_scaled))

5/5 [==============================] - 0s 744us/step - loss: 0.1548 - mse: 0.1548
[0.15477198362350464, 0.15477198362350464]

5/5 [==============================] - 0s 741us/step - loss: 0.1665 - mse: 0.1665
[0.16654819250106812, 0.16654819250106812]


As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [46]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
RMSE_final = mean_squared_error(y_test, y_test_pred, squared = False)
RMSE_final

30915.59986780489

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.